# 📘 LSTM Surrogate Model for Biogas Prediction

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/benmola/OpenAD-lib/blob/main/notebooks/03_LSTM_Prediction.ipynb)

This notebook demonstrates using **Long Short-Term Memory (LSTM)** networks as surrogate models for fast biogas production prediction.

---

## 📚 References

- **LSTM for AD Processes**: [Murali et al. (2025) - LAPSE](https://psecommunity.org/LAPSE:2025.0213)

## 🔬 LSTM Background

LSTM networks are a type of recurrent neural network (RNN) designed to learn long-term dependencies in sequential data.

### LSTM Cell Equations

**Forget Gate:**
$$f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)$$

**Input Gate:**
$$i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)$$
$$\tilde{C}_t = \tanh(W_C \cdot [h_{t-1}, x_t] + b_C)$$

**Cell State Update:**
$$C_t = f_t \odot C_{t-1} + i_t \odot \tilde{C}_t$$

**Output Gate:**
$$o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o)$$
$$h_t = o_t \odot \tanh(C_t)$$

### Why LSTM for AD?

1. **Temporal Dependencies**: Biogas production depends on past substrate loading
2. **Non-linear Dynamics**: Captures complex microbial interactions
3. **Fast Inference**: ~1000x faster than mechanistic models
4. **Robust to Noise**: Handles measurement variability

## 1️⃣ Setup (Google Colab)

In [ ]:
# Install OpenAD-lib with ML support (uncomment for Colab)
# !pip install "git+https://github.com/benmola/OpenAD-lib.git#egg=openad_lib[ml]"

import sys
import os

IN_COLAB = 'google.colab' in sys.modules

if not IN_COLAB:
    sys.path.append(os.path.join(os.getcwd(), '..', 'src'))

print(f"Running in Colab: {IN_COLAB}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

from openad_lib.models.ml import LSTMModel

print("✅ All imports successful!")

## 2️⃣ Load and Prepare Data

We'll use time-series data from a biogas plant with:
- **Inputs**: Feedstock composition (Maize, Chicken Litter, etc.)
- **Output**: Total biogas production

In [ ]:
# Load data
if IN_COLAB:
    !wget -q https://raw.githubusercontent.com/benmola/OpenAD-lib/main/src/openad_lib/data/sample_LSTM_timeseries.csv
    data_path = 'sample_LSTM_timeseries.csv'
else:
    base_path = os.path.dirname(os.getcwd())
    data_path = os.path.join(base_path, 'src', 'openad_lib', 'data', 'sample_LSTM_timeseries.csv')

data = pd.read_csv(data_path).dropna()
print(f"📊 Loaded {len(data)} samples")
print(f"   Columns: {list(data.columns)}")
data.head()

In [ ]:
# Define features and target
features = ['Maize', 'Wholecrop', 'Chicken Litter', 'Lactose', 'Apple Pomace', 'Rice bran']
target = 'Total_Biogas'

X = data[features].values
y = data[target].values

# 80/20 Train-Test Split
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print(f"Training samples: {len(X_train)}")
print(f"Testing samples: {len(X_test)}")

## 3️⃣ Build and Train LSTM Model

The `LSTMModel` class handles:
- Data normalization (StandardScaler)
- Sequence creation for time-series
- PyTorch model training

In [ ]:
# Initialize LSTM model
lstm = LSTMModel(
    input_dim=len(features),
    hidden_dim=24,
    output_dim=1,
    num_layers=1,
    dropout=0.1,
    learning_rate=0.001
)

print("🚀 Training LSTM model...")
lstm.fit(X_train, y_train, epochs=50, batch_size=4, verbose=True)

## 4️⃣ Evaluate Model

Metrics:
- **RMSE**: Root Mean Squared Error
- **MAE**: Mean Absolute Error  
- **R²**: Coefficient of Determination

In [ ]:
# Evaluate
metrics = lstm.evaluate(X_test, y_test)

print("📈 LSTM Test Metrics:")
print(f"   RMSE: {metrics['rmse']:.2f}")
print(f"   MAE:  {metrics['mae']:.2f}")
print(f"   R²:   {metrics['r2']:.3f}")

In [ ]:
# Predictions
y_pred = lstm.predict(X_test)

# Plot results
plt.style.use('bmh')
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(y_test, label='Actual', color='#2E86C1', linewidth=2, alpha=0.8)
ax.plot(y_pred, label='LSTM Prediction', color='#E67E22', linewidth=2, linestyle='--')

ax.set_xlabel('Time Step', fontsize=14, fontweight='bold')
ax.set_ylabel('Biogas Production', fontsize=14, fontweight='bold')
ax.set_title('LSTM Biogas Prediction (Test Set)', fontsize=16, pad=20)
ax.legend(fontsize=12)
ax.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## 📝 Summary

This notebook demonstrated:

1. **LSTM Architecture** for time-series prediction
2. **Training** with feedstock composition data
3. **Evaluation** using RMSE, MAE, R² metrics
4. **Visualization** of predictions vs actual values

### Next Steps

- Try [Multi-Task GP](04_MTGP_Prediction.ipynb) for uncertainty quantification
- Compare with [ADM1 mechanistic model](01_ADM1_Tutorial.ipynb)
- Explore [MPC Control](05_MPC_Control.ipynb)